# Configuration of Jupyter Notebooks at PDC

Jupyter Notebooks have become a mainstream tool for interactive computing in every field of science. Jupyter is slowly making its way into HPC, and PDC has recently started offering access to Jupyter on the Tegner cluster.

In this lesson, we will cover: 
- Necessary configuration steps needed to run Jupyter on Tegner@PDC
- How to start a Jupyter server on a compute node

In following lessons, we will cover:
- Basics of Jupyter: What is it? How can we use it?
- How to launch jobs via Jupyter to the Slurm scheduler
- How to monitor and analyze simulation results generated by other software on the fly in Jupyter
- Some parallel computing concepts in Python using `multiprocessing`, `ipyparallel` and `mpi4py`

## First fetch the sources

Log in to Tegner, create a new directory and clone the repository with the lesson material:

```bash
$ ssh <username>@tegner.pdc.kth.se
$ cd /cfs/klemming/nobackup/<initial>/<username>
$ mkdir prace-jupyter
$ cd prace-jupyter
$ git clone https://github.com/PDC-support/jupyter-notebook.git
```

## Setting up Jupyter on Tegner

To connect from your browser to a Jupyter Notebook server running on a Tegner compute node you need to follow a few steps.
First make sure that the connection between your browser and the Jupyter server running at PDC is secure, by:
1. Generating a Jupyter configuration file
2. Setting a strong Jupyter password
3. Setting up a self-signed SSL certificate to enable https

### 1. Jupyter configuration

- Load module ``anaconda/py36/5.0.1`` and activate the ``prace`` environment.

```bash
$ module load anaconda/py36/5.0.1
$ source activate prace
```

- Generate the configuration file `$HOME/.jupyter/jupyter_notebook_config.py`: 

```bash
$ jupyter notebook --generate-config
```

- Generate the configuration file `$HOME/.ipython/profile_default/ipython_config.py`: 

```bash
$ ipython profile create
```

The configuration file will have many lines with possible configuration options, but all of them will be commented out initially.

### 2. Setting a Jupyter password
+ Create your Jupyter password 

```
$ jupyter notebook password
```

+ The entered password will be saved in hashed form in the file `$HOME/.jupyter/jupyter_notebook_config.json`.

### 3. Generate a SSL certificate

+ Generate a self-signed certificate with `openssl` in your `Private` directory: 

```bash
$ cd $HOME/Private
$ openssl req -x509 -nodes -days 30 -newkey rsa:2048 -keyout mykey.key -out mycert.pem
```

+ Add information on the certificate in your Jupyter config file `$HOME/.jupyter/jupyter_notebook_config.py`:

```
c.NotebookApp.certfile = u'/afs/pdc.kth.se/home/<initial>/<username>/Private/mycert.pem'
c.NotebookApp.keyfile = u'/afs/pdc.kth.se/home/<initial>/<username>/Private/mykey.key'
```

+ Go back to your jupyter-notebook repository

```
$ cd /cfs/klemming/nobackup/<initial>/<username>/prace-jupyter/jupyter-notebook
```

## Using Jupyter on Tegner

There are two ways of running Jupyter on Tegner:
- On a login or transfer node (for submitting jobs and for lightweight calculations)
- On a compute node via a batch script (for parallel and/or memory-demanding calculations)

### Lightweight use on login/transfer node

- Before starting Jupyter, make sure you have set the Jupyter password and generated an SSL certificate and listed it in your Jupyter config file (see above). 
- Start Jupyter notebook with port, host IP, and secure protocol

```bash
$ ipnport=$(shuf -i8000-9999 -n1)
$ ipnip=$(hostname -i)
$ jupyter-notebook --no-browser --port=$ipnport --ip=$ipnip
```

- Jupyter notebook will print output like:

```
The Jupyter Notebook is running at:
https://<IP.address>:<port>/
```

- Copy the **https** link to your browser. Your browser will likely complain that your self-signed certificate is not secure or not recognized. Tell it to proceed anyways.


- The notebook is ready for use after entering your password!

### Batch script

- Before starting Jupyter, make sure you have set the Jupyter password and generated an SSL certificate and listed it in your Jupyter config file (see above).

- Copy/paste this batch script to a file `submit_jupyter.bash`:

```bash
#!/bin/bash
#SBATCH -A edu18.prace
#SBATCH --reservation prace-2018-10-26
#SBATCH --nodes 1
#SBATCH --time 1:00:00
#SBATCH --job-name jupyter
#SBATCH --output jupyter-log-%J.txt

module load anaconda/py36/5.0.1
source activate prace

XDG_RUNTIME_DIR=""
ipnport=$(shuf -i8000-9999 -n1)
ipnip=$(hostname -i)

jupyter-notebook --no-browser --port=$ipnport --ip=$ipnip
```

- Submit the batch script by 

```bash
$ sbatch submit_jupyter.bash
```

- Check the output in `jupyter-log-%J.txt`:
    - Copy/paste the URL `https://<IP.address>:<port>/` provided in the output file into your browser. 

### Batch script

- Before starting Jupyter, make sure you have set the Jupyter password and generated an SSL certificate and listed it in your Jupyter config file (see above).

- Copy/paste this batch script to a file `submit_jupyter.bash`:

```bash
#!/bin/bash
#SBATCH -A edu18.prace
#SBATCH --nodes 1
#SBATCH --time 1:00:00
#SBATCH --job-name tunnel
#SBATCH --output jupyter-log-%J.txt

module load anaconda/py36/5.0.1
source activate prace

XDG_RUNTIME_DIR=""
ipnport=$(shuf -i8000-9999 -n1)
ipnip=$(hostname -i)

echo -e "
    Open a browser on your local machine to the following address
    ------------------------------------------------------------------
    https://$ipnip:$ipnport/
    ------------------------------------------------------------------
    "

jupyter-notebook --no-browser --port=$ipnport --ip=$ipnip
```

- Submit the batch script by 

```bash
$ sbatch submit_jupyter.bash
```

- Check the instructions in `jupyter-log-%J.txt`:
    - Copy/paste the URL `https://<IP.address>:<port>/` provided in the output file into your browser. 